## Importamos las dependencias

In [ ]:
!pip install langchain_core
!pip install langchain
!pip install langchain_openai
!pip install openai

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain.agents import create_react_agent, AgentExecutor
from langchain_openai.chat_models import ChatOpenAI
from langchain.tools import tool
import json
import os

os.environ["OPENAI_API_KEY"] = ""
llm = ChatOpenAI(model_name="gpt-4o")


# Creando nuestro primer Langchain tool

In [ ]:
@tool
def return_weather(json_input) -> str:
    """
    Return the current weather in a location

    Example of input JSON:
    {
        "location": "Location where you want to get the weather",
    }
    """

    # Perform similarity search on the precision collection
    parsed_json = json.loads(json_input)

    location = parsed_json.get("location", None)

    if location is None:
        return

    weather_info = f"El tiempo en {location} es cálido y soleado"
    return weather_info

## Creamos el PromptTemplate del agente ReAct

In [ ]:
prompt = PromptTemplate(
        input_variables=["input", "tool_names", "agent_scratchpad"],
        template="""
        You are an intelligent assistant that always thinks in English, capable of providing detailed responses related to companies using the tools: {tool_names}.
        Here are the available tools: {tools}

        If you need to call a tool, respond using this exact format:

        Thought: [Explain your reasoning]
        Action: Tool to call
        Action Input: Tool input in JSON format with no additional quotes around the entire input

        If a tool response is received:
        - Extract all the relevant information to give a proper answer to user query.
        - Provide a final answer in the following format:

        Final Answer: [Your final response to the query: {input}. Include all relevant information with details.Answer always in spanish]

        Query: {input}

        {agent_scratchpad}
        """
        )

In [ ]:
tools = [return_weather]

In [ ]:
react_agent = create_react_agent(llm=llm, tools=tools, prompt=prompt)

agent_executor = AgentExecutor.from_agent_and_tools(
        react_agent, tools=tools, handle_parsing_errors=True
    )

In [ ]:
agent_response = agent_executor.invoke(
                {
                    "input": "Cual es el tiempo en mónaco"
                }
            )
print(agent_response)

{'input': 'Cual es el tiempo en mónaco', 'output': 'El tiempo en Mónaco es cálido y soleado.'}
